# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [ ]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
from newsapi import NewsApiClient
%matplotlib inline
load_dotenv()

In [ ]:
# Read your api key environment variable
api_key = os.getenv('newsapi')


In [ ]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)


In [ ]:
# Fetch the Bitcoin news articles
## get_everything
BTC_news = newsapi.get_everything(q='bitcoin', language="en", sort_by="relevancy")
#BTC_news
print(f"Total articles about Bitcoin News: {BTC_news['totalResults']}")

In [ ]:
# Fetch the Ethereum news articles
ETH_news = newsapi.get_everything(q="ethereum", language="en", sort_by ="relevancy")

print(f"Total articles about Ethereum: {ETH_news['totalResults']}")

In [ ]:
# Create the Bitcoin sentiment scores DataFrame
BTC_sent = []
for article in BTC_news["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        BTC_sent.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
BTC = pd.DataFrame(BTC_sent)

In [ ]:
# Create the Ethereum sentiment scores DataFrame
ETH_sent = []
for article in ETH_news["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        ETH_sent.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
ETH = pd.DataFrame(ETH_sent)

In [ ]:
# Describe the Bitcoin Sentiment
BTC.describe()

In [ ]:
# Describe the Ethereum Sentiment
ETH.describe()
#ETH_Summary = ETH.describe()
#ETH_Summary.loc["negative"]

### Questions:

Q: Which coin had the highest mean positive score?

A: Bitcoin had the higher mean positive score

Q: Which coin had the highest compound score?

A: Ethereum

Q. Which coin had the highest positive score?

A: Bitcoin 

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [ ]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [ ]:
# Instantiate the lemmatizer
lemmatizer = WordNetLemmatizer()
# Create a list of stopwords
sw = list(set(stopwords.words('english')))
# Expand the default stopwords list if necessary
sw.append(["the", "as", "if"])

In [ ]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    
    # Remove the punctuation from text
    # list(filter) so that we can have the list filter the punctuations.
    words =  word_tokenize(text)
    words = list(filter(lambda T:T not in punctuation, words))
    words = list(filter(lambda T:T.lower(), words))
    words = [lemmatizer.lemmatize(word) for word in words]
    tokens = list(filter(lambda T:T.lower() not in sw, words))
    # Convert the words to lowercase
    
    # Remove the stop words
        # Create a tokenized list of the words
    # Lemmatize words into root words
    
    return tokens

In [ ]:
# Create a new tokens column for Bitcoin
BTC["new_tokens"] = BTC.text.apply(tokenizer)
BTC

In [ ]:
# Create a new tokens column for Ethereum
ETH["new_tokens"] = ETH.text.apply(tokenizer)
ETH

---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [ ]:
from collections import Counter
from nltk import ngrams

In [ ]:
# Generate the Bitcoin N-grams where N=2
n=2
grams = ngrams(tokenizer(BTC.text.str.cat()), n)
Counter(grams).most_common(10)

In [ ]:
# Generate the Ethereum N-grams where N=2
n=2
grams = ngrams(tokenizer(ETH.text.str.cat()), n)
Counter(grams).most_common(10)

In [ ]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [ ]:
# Use token_count to get the top 10 words for Bitcoin
BTC_ALL = tokenizer(BTC.text.str.cat())
token_count(BTC_ALL, 10)

In [ ]:
# Use token_count to get the top 10 words for Ethereum
ETH_ALL = tokenizer(ETH.text.str.cat())
token_count(ETH_ALL, 10)

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [ ]:
# Generate the Bitcoin word cloud
wc = WordCloud().generate(BTC.text.str.cat())
plt.imshow(wc)

In [ ]:
# Generate the Ethereum word cloud
wc = WordCloud().generate(ETH.text.str.cat())
plt.imshow(wc)

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [ ]:
import spacy
from spacy import displacy

In [ ]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [ ]:
# Concatenate all of the Bitcoin text together
doc = nlp(BTC.text.str.cat())

In [ ]:
# Run the NER processor on all of the text
# Render the visualization

displacy.render(doc, style='ent')

In [ ]:
# List all Entities
print([ent.text for ent in doc.ents if ent.label_ == 'ORG'])

---

### Ethereum NER

In [ ]:
# Concatenate all of the Ethereum text together
doc = nlp(ETH.text.str.cat())

In [ ]:
# Run the NER processor on all of the text
displacy.render(doc, style='ent')

# Add a title to the document


In [ ]:
# Render the visualization
displacy.render(doc, style='ent')

In [ ]:
# List all Entities
print([ent.text for ent in doc.ents if ent.label_ == 'ORG'])

---